In [2]:
import random
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations, groupby
import time
from tqdm import tqdm
from networkx.algorithms import tree

In [4]:
def gnp_random_connected_graph(num_of_nodes: int,
                               completeness: int,
                               directed: bool = False,
                               draw: bool = False):
    """
    Generates a random graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted (in case of undirected graphs)
    """
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    edges = combinations(range(num_of_nodes), 2)
    G.add_nodes_from(range(num_of_nodes))
    
    for _, node_edges in groupby(edges, key = lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        if random.random() < 0.5:
            random_edge = random_edge[::-1]
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < completeness:
                G.add_edge(*e)
                
    for (u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(-5, 20)
                
    if draw:
        plt.figure(figsize=(10,6))
        if directed:
            # draw with edge weights
            pos = nx.arf_layout(G)
            nx.draw(G,pos, node_color='lightblue',
                    with_labels=True,
                    node_size=500, 
                    arrowsize=20, 
                    arrows=True)
            labels = nx.get_edge_attributes(G,'weight')
            nx.draw_networkx_edge_labels(G, pos,edge_labels=labels)
            
        else:
            nx.draw(G, node_color='lightblue',
                with_labels=True,
                node_size=500)
    return G


In [5]:
def kruskals_algorithm(graph):
    """
    Kruskal's algorithm to find the minimum spanning tree for a connected weighted graph
    """
    graph_edges = graph.edges(data=True) # getting edges with their weight
    sorted_edges = sorted(graph_edges, key= lambda x: x[2].get('weight')) # sorted edges from the smalles to the biggest weight
    dict_of_connected_nodes = {} # dictionary for nodes as a keys and list with connected nodes with them as a value
    connected_nodes = [] # list for saving nodes, that are already connected
    edges = [] # resulting list with the minimum spanning tree
    for edge in sorted_edges:
        if not edge[0] in connected_nodes or not edge[1] in connected_nodes: # checking if one of the nodes are not connected
            if not edge[0] in connected_nodes and not edge[1] in connected_nodes: # checking for both
                dict_of_connected_nodes[edge[0]] = [edge[0], edge[1]] # connect two nodes with each other
                dict_of_connected_nodes[edge[1]] = [edge[0], edge[1]]
            else:
                if not dict_of_connected_nodes.get(edge[0]): # another case when first node is isolated
                    dict_of_connected_nodes[edge[1]].append(edge[0])
                    dict_of_connected_nodes[edge[0]] = dict_of_connected_nodes[edge[1]]
                else: # another case when second node is isolated
                    dict_of_connected_nodes[edge[0]].append(edge[1])
                    dict_of_connected_nodes[edge[1]] = dict_of_connected_nodes[edge[0]]
            edges.append(edge)
            connected_nodes.append(edge[0])
            connected_nodes.append(edge[1])
    for i in sorted_edges:
        if i[0] in dict_of_connected_nodes[i[0]] and i[1] not in dict_of_connected_nodes[i[0]]: # connecting different sets of nodes with each other
            edges.append(i)
    return edges


In [8]:
NUM_OF_ITERATIONS = 10
time_taken_my_algo = 0
time_taken_pc_algo = 0
list_with_nums = [10, 20, 50, 100, 200, 500]
results_my = []
results_pc = []

for i in list_with_nums:
    for j in tqdm(range(NUM_OF_ITERATIONS)):
        G = gnp_random_connected_graph(i, 0.4, False)
        
        start = time.time()
        kruskals_algorithm(G)
        end = time.time()
        time_taken_my_algo += end - start

        start = time.time()
        tree.minimum_spanning_tree(G, algorithm="kruskal")
        end = time.time() 
        time_taken_pc_algo += end - start

    results_my.append(time_taken_my_algo / NUM_OF_ITERATIONS)
    results_pc.append(time_taken_pc_algo / NUM_OF_ITERATIONS)

for index in range(len(list_with_nums)):
    print(f"Size - {list_with_nums[index]}: My realization - {results_my[index]} | Networkx algorithms - {results_pc[index]}")

100%|██████████| 10/10 [00:07<00:00,  1.30it/s]

Size - 10: My realization - 4.08172607421875e-05 | Networkx algorithms - 9.512901306152344e-05
Size - 20: My realization - 0.00017137527465820311 | Networkx algorithms - 0.00034558773040771484
Size - 50: My realization - 0.001046156883239746 | Networkx algorithms - 0.001530933380126953
Size - 100: My realization - 0.006176972389221191 | Networkx algorithms - 0.0057702302932739254
Size - 200: My realization - 0.04549431800842285 | Networkx algorithms - 0.025313591957092284
Size - 500: My realization - 0.5803280353546143 | Networkx algorithms - 0.1470111131668091
